# Generating melody *and* harmony using LSTMs

Again, using data from the Weimar Jazz Database.

First, let's import the `jazzaiexperiments` module:

In [1]:
import jazzaiexperiments

Using TensorFlow backend.


Next, we'll pick a tune:

In [2]:
tune_name = "ColemanHawkins_BodyAndSoul_FINAL"

### Approach #1: Add underlying harmony directly to note events

In [3]:
import mido
import pandas as pd

In [4]:
# Testing + building jazzaiexperiments functions
# Based on lstm_train_on_midi_input()

from jazzaiexperiments import *

# Set variables (that will be passed in as arguments)
tune_name = "ColemanHawkins_BodyAndSoul_FINAL"
midi_data_dir = "../data/midi/quantized/"
checkpoints_data_dir = "../data/models/"
seq_length = 10
num_epochs = 1

# Create note events
input_filepath = midi_create_input_filepath(tune_name, midi_data_dir)
midi_track = midi_load_melody_from_file(input_filepath)
note_pairs = midi_extract_note_pairs(midi_track)
note_pairs = midi_normalize_velocities(note_pairs, interval=10)
note_events = midi_create_note_events(note_pairs)
print("Created note events from {}".format(input_filepath))

# Format note data to feed into network
note_set = midi_create_note_set(note_events)
seqs_input, seqs_output = midi_split_subsequences(note_events,
                                                  seq_length=seq_length)
num_seqs = len(seqs_input)
seq_length = len(seqs_input[0])
num_unique_notes = len(note_set)
x, y = midi_format_for_lstm(seqs_input, seqs_output,
                            num_seqs=num_seqs,
                            seq_length=seq_length,
                            num_unique_notes=num_unique_notes)
print("Formatted note data ({} seqs of length {}, "
      "{} unique notes)".format(num_seqs, seq_length, num_unique_notes))

Created note events from ../data/midi/quantized/ColemanHawkins_BodyAndSoul_FINAL.mid
Formatted note data (625 seqs of length 10, 365 unique notes)


In [6]:
def midi_note_event_to_dict(note):
    note_events_keys = midi_get_note_event_keys()
    return dict((note_events_keys[i], note[i]) for i,_ in enumerate(note))

midi_note_event_to_dict(note_events[0])

{'noteoff_time': 16170,
 'noteon_pitch': 51,
 'noteon_time': 76230,
 'noteon_velocity': 100}

In [7]:
[msg for msg in midi_track if "note" not in msg.type]

[<meta message track_name name='Melody' time=0>,
 <meta message set_tempo tempo=631578 time=0>,
 <meta message key_signature key='Db' time=0>,
 <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>,
 <meta message end_of_track time=0>]

In [8]:
midi_track[5]

<message note_off channel=0 note=51 velocity=100 time=16170>

In [9]:
mf = mido.MidiFile(input_filepath)
mf.ticks_per_beat

27720

In [10]:
def calculate_note_times_seconds(input_filepath):
    midi_file = mido.MidiFile(input_filepath)
    midi_track = midi_load_melody_from_file(input_filepath)
    tempo = midi_track[1].tempo
    # ppq = midi_track[3].clocks_per_click
    # n32 = midi_track[3].notated_32nd_notes_per_beat
    ppq = midi_file.ticks_per_beat
    note_times = [mido.tick2second(msg.time, ppq, tempo) for msg in midi_track if "note" in msg.type]
    note_times_summed = []
    for i, t in enumerate(note_times):
        note_times_summed.append(sum(note_times[:i]) + t)
    return note_times_summed

calculate_note_times_seconds(input_filepath)

[1.7368395,
 2.10526,
 2.10526,
 2.315786,
 2.315786,
 3.3157845000000004,
 3.3157845000000004,
 3.473679,
 3.473679,
 3.6315735,
 3.6315735,
 3.789468,
 3.789468,
 3.8947309999999997,
 3.8947309999999997,
 3.9999939999999996,
 3.9999939999999996,
 4.8947294999999995,
 4.8947294999999995,
 5.210518499999999,
 5.210518499999999,
 5.526307499999999,
 5.526307499999999,
 5.999990999999999,
 5.999990999999999,
 7.105252499999999,
 7.105252499999999,
 7.263146999999999,
 7.263146999999999,
 7.421041499999999,
 7.421041499999999,
 7.578936,
 7.578936,
 7.894724999999999,
 7.894724999999999,
 8.210514,
 8.842092,
 9.052617999999999,
 9.052617999999999,
 9.263143999999999,
 9.263143999999999,
 9.684195999999998,
 9.684195999999998,
 9.894721999999998,
 9.894721999999998,
 10.105247999999998,
 10.105247999999998,
 10.315751215800864,
 10.315751215800864,
 11.210486715800863,
 11.210486715800863,
 11.368381215800863,
 11.368381215800863,
 12.105222215800863,
 12.105222215800863,
 12.210485215800

In [11]:
def midi_add_harmony_to_notes(note_events):
    # TODO
    # So we'll need the note events but also the chordal info. From a CSV export from the database itself??
    pass

In [12]:
def lstm_train_with_harmony():
    # Set variables (that will be passed in as arguments)
    tune_name = "ColemanHawkins_BodyAndSoul_FINAL"
    midi_data_dir = "../data/midi/quantized/"
    checkpoints_data_dir = "../data/models/"
    seq_length = 10
    num_epochs = 1

    # Create note events
    input_filepath = midi_create_input_filepath(tune_name, midi_data_dir)
    midi_track = midi_load_melody_from_file(input_filepath)
    note_pairs = midi_extract_note_pairs(midi_track)
    note_pairs = midi_normalize_velocities(note_pairs, interval=10)
    note_events = midi_create_note_events(note_pairs)
    print("Created note events from {}".format(input_filepath))
    
    # Add harmony!!
    # TODO

    # Format note data to feed into network
    note_set = midi_create_note_set(note_events)
    seqs_input, seqs_output = midi_split_subsequences(note_events,
                                                      seq_length=seq_length)
    num_seqs = len(seqs_input)
    seq_length = len(seqs_input[0])
    num_unique_notes = len(note_set)
    x, y = midi_format_for_lstm(seqs_input, seqs_output,
                                num_seqs=num_seqs,
                                seq_length=seq_length,
                                num_unique_notes=num_unique_notes)
    print("Formatted note data ({} seqs of length {}, "
          "{} unique notes)".format(num_seqs, seq_length, num_unique_notes))
    
lstm_train_with_harmony()

Created note events from ../data/midi/quantized/ColemanHawkins_BodyAndSoul_FINAL.mid
Formatted note data (625 seqs of length 10, 365 unique notes)


### Training + generation from previous notebook

Let's build and train an LSTM model from a MIDI file:

In [13]:
# model, note_events, input_filepath = jazzaiexperiments.lstm_train_on_midi_input(tune_name,
#                                                                                 "../data/midi/quantized/",
#                                                                                 "../data/models/",
#                                                                                 num_epochs=10)
# model.summary()

This is some example output for 10 epochs (it's not very good, and basically gets stuck in a two-to-three-note loop): https://soundcloud.com/usdivad/jazz-ai-experiments-lstm-single-melody-coleman-hawkins-body-and-soul-10-epochs

We've actually trained this model before, so let's load some existing weights (from 100 epochs of training) into our current model:

In [14]:
# model = jazzaiexperiments.lstm_load_weights(model, "../data/models/weights_ColemanHawkins_BodyAndSoul_FINAL_20170702124647869274_99_0.8517.hdf5")
# model.summary()

Note that we can also do this by passing in a `weights_filepath` argument into the `lstm_train_on_midi_input` function when creating the model.

And let's generate some output!

In [15]:
# notes_out = jazzaiexperiments.lstm_generate_midi_output(model, note_events,
#                                                         num_notes_to_generate=100,
#                                                         random_seed=False,
#                                                         add_seed_to_output=True,
#                                                         tune_name=tune_name,
#                                                         midi_source_filepath=input_filepath,
#                                                         data_dir="../data/output")
# notes_out[:20]

Example output for 100 epochs: https://soundcloud.com/usdivad/jazz-ai-experiments-lstm-single-melody-coleman-hawkins-body-and-soul-100-epochs